---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd

def get_list_of_university_towns():

    s = sorted([v for k,v in states.items()])
    s.remove('American Samoa')
    s.remove('District of Columbia')
    s.remove('Northern Mariana Islands')
    s.remove('Puerto Rico')
    s.remove('Virgin Islands')
    s.remove('Guam')
    s.remove('National')
    
    uni_towns = pd.read_table("university_towns.txt", names=["State", "RegionName"])
    uni_towns["State"] = uni_towns["State"].map(lambda x: x.split('[')[0])
    column1 = uni_towns['State'].tolist()

    list_state_dict = []
    temp_state = ''

    for item in column1:
        if item in s:
            temp_state = item
        else:
            list_state_dict.append({'RegionName':item, 'State': temp_state})

    uni_states = pd.DataFrame(list_state_dict)
    uni_states = uni_states.sort_index(axis=1, ascending=False) 
    uni_states["RegionName"] = uni_states["RegionName"].map(lambda x: x.split(' (')[0])
    return uni_states

get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():  
    
    df = pd.read_excel('gdplev.xls', header=None, skiprows=220, parse_cols=(4,5), 
                   names=['Year_Q', 'GDP in billions'])
  
    for index in df.index:
         
        if df.loc[index, 'GDP in billions'] > df.loc[index+1, 'GDP in billions'] > df.loc[index+2, 'GDP in billions']:
            m = index
            while df.loc[m, 'GDP in billions'] > df.loc[m+1, 'GDP in billions']:
                m += 1
            if df.loc[m, 'GDP in billions'] < df.loc[m+1, 'GDP in billions'] < df.loc[m+2, 'GDP in billions']:
                year_quarter = df.loc[index, 'Year_Q']
                break 
    
    return year_quarter

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', header=None, skiprows=220, parse_cols=(4,5), 
                   names=['Year_Q', 'GDP in billions'])
  
    for index in df.index:
         
        if df.loc[index, 'GDP in billions'] > df.loc[index+1, 'GDP in billions'] > df.loc[index+2, 'GDP in billions']:
            m = index
            while df.loc[m, 'GDP in billions'] > df.loc[m+1, 'GDP in billions']:
                m += 1
            if df.loc[m, 'GDP in billions'] < df.loc[m+1, 'GDP in billions'] < df.loc[m+2, 'GDP in billions']:
                year_quarter = df.loc[m+2, 'Year_Q']
                break 
       
    return year_quarter

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', header=None, skiprows=220, parse_cols=(4,5), 
                   names=['Year_Q', 'GDP in billions'])
    
    recession_period = df.loc[34:38]
    
    rec_bottom = recession_period['GDP in billions'].min()
    
    rec_bottom_yearq = recession_period['Year_Q'][recession_period['GDP in billions']==rec_bottom].tolist()
    
    return rec_bottom_yearq[0]

get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    raw_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    raw_df['State'].replace(states, inplace=True)
    
    # seperate time columns and convert their names to datetime
    tdf = raw_df[raw_df.columns[10:]].rename(columns=pd.to_datetime)

    # find the columns in the period 2000-2016
    cols = tdf.columns
    sel_cols = cols[(cols >= '2000')]

    # select the columns, resample on columns for mean, rename the columns as year and quarter 
    mdf = tdf[sel_cols].resample('Q',axis=1).mean().rename(columns=lambda x:'{:}q{:}'.format(x.year, x.quarter))

    # reattach non-time columns 'State' and 'RegionName' and set them as index
    mdf['State'] = raw_df['State']
    mdf['RegionName'] = raw_df['RegionName'] 
    mdf.set_index(['State', 'RegionName'], inplace=True)
    
    return mdf

convert_housing_data_to_quarters()                    

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3,2016q4
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.189333e+05,5.218000e+05,5.235667e+05,5.325000e+05,5.480000e+05,5.614667e+05,5.726333e+05,5.853333e+05,6.023000e+05,6.209333e+05
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,5.089667e+05,5.178333e+05,5.269333e+05,5.355667e+05,5.450333e+05,5.568667e+05,5.682667e+05,5.814333e+05,5.901333e+05,6.045333e+05
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.901667e+05,1.935667e+05,1.949000e+05,1.989000e+05,2.015333e+05,2.027000e+05,2.040333e+05,2.058333e+05,2.078667e+05,2.123000e+05
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.159667e+05,1.166333e+05,1.167667e+05,1.178000e+05,1.205333e+05,1.221667e+05,1.238333e+05,1.261000e+05,1.305000e+05,1.325667e+05
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.675000e+05,1.705333e+05,1.733333e+05,1.765000e+05,1.805000e+05,1.851667e+05,1.890667e+05,1.931667e+05,1.985667e+05,2.028333e+05
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.716000e+05,1.746333e+05,1.776667e+05,1.803333e+05,1.852333e+05,1.887667e+05,1.922667e+05,1.961333e+05,2.025667e+05,2.066333e+05
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.810000e+05,4.859667e+05,4.932000e+05,5.036000e+05,5.127667e+05,5.208333e+05,5.277000e+05,5.327000e+05,5.425000e+05,5.500000e+05
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.071333e+05,1.097667e+05,1.124667e+05,1.201667e+05,1.280667e+05,1.343000e+05,1.398333e+05,1.455667e+05,1.554000e+05,1.592000e+05
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,7.004667e+05,7.178000e+05,7.375000e+05,7.643000e+05,7.874667e+05,8.051667e+05,8.247333e+05,8.392667e+05,8.349667e+05,8.432333e+05


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    '''Hypothesis: University towns have their mean housing prices less effected 
    by recessions. Run a t-test to compare the ratio of the mean price of houses 
    in university towns the quarter before the recession starts compared to the 
    recession bottom. (price_ratio=quarter_before_recession/recession_bottom)'''
    
    # calling mean dataFrame of towns, calculate price ratio = 2008q2/2009q2 and add column
    mdf = convert_housing_data_to_quarters()
    mdf['Price Ratio']=mdf['2008q2'].div(mdf['2009q2'])
    mdf = mdf.reset_index()
    
    #calling university towns dataFrame
    udf = get_list_of_university_towns()
    
    # merging both dataFrames on the State and RegionNames
    merge_df = pd.merge(mdf, udf, how='left', left_on=['State', 'RegionName'], right_on=['State','RegionName'] , indicator=True)
    
    # slicing the merged dataFrame for only shared RegionNames, calling 'Price Ratio' column
    # utm = university town means 
    utm = merge_df[merge_df['_merge']=='both']
    utm = utm['Price Ratio'].dropna()
    utm_mean = utm.mean()
    
    # slicing the merged dataFrame for non-shared RegionNames, calling 'Price Ratio' column
    # otm = other town means
    otm = merge_df[merge_df['_merge']=='left_only']
    otm = otm['Price Ratio'].dropna()
    otm_mean = otm.mean()
   
    #t-test for the Price Ratio means
    pv = ttest_ind(utm, otm, nan_policy='omit')
    
    def different(pvalue):
        if pvalue < 0.01:
            return True
        else:
            return False
    
    def better(mean1, mean2):
        if mean1 < mean2:
            return "university town"
        else:
            return "non-university town"
    
    answer = (different(pv[1]), pv[1], better(utm_mean, otm_mean))
    
    return answer

run_ttest()


(True, 0.005182164348468756, 'university town')